In [1]:
# 1 Importing Dependencies
 
import pandas as pd
import requests
import sys
from pymongo import MongoClient
sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import nba_rapidapi_key


In [2]:
# 2 Gathering list of team ids needed to call API

# Establishing MongoDB Connection
mongo = MongoClient('mongodb://localhost:27017/')
db = mongo['nba_data2']
collection = db['teams']

# Get distinct 'id' values
nba_team_ids = collection.distinct('team.id')

# Print the length of the list, should be 30 teams
print(f"There are {len(nba_team_ids)} teams")

# Close the MongoDB connection
mongo.close()

nba_team_ids

There are 30 teams


[1,
 2,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 38,
 40,
 41]

In [3]:
# 3 Creating collection for team stats from nba-api

# Establishing MongoDB Connection
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["nba_data2"]
collection = db["team_stats"]

# Dropping collection if already exists
if "team_stats" in db.list_collection_names():
    db.drop_collection("team_stats")
    print("Existing 'team_stats' collection dropped.")

# Setting up API call structure for NBA-API.com 
url = "https://api-nba-v1.p.rapidapi.com/teams/statistics"

headers = {
	"X-RapidAPI-Key": nba_rapidapi_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

# Calling API looping through team_ids
for team_id in nba_team_ids:
    querystring = {"id":team_id,"season":"2023"}
    response = requests.get(url, headers=headers, params=querystring)

    # Splitting out each team's stats from the response and putting into MongoDB collection "team_stats"
    if response.status_code == 200:
        response_data = response.json()
        for stats in response_data['response']:
            team_document = {"season": 2023, "team_id": team_id, "stats": stats}
            collection.insert_one(team_document)
            print(f"Inserted stats from team {team_id} into MongoDB")
    else:
        print(f"Failed to fetch data for team {team_id}, status code: {response.status_code}")
        
# Close the MongoDB connection
mongo.close()

Inserted stats from team 1 into MongoDB
Inserted stats from team 2 into MongoDB
Inserted stats from team 4 into MongoDB
Inserted stats from team 5 into MongoDB
Inserted stats from team 6 into MongoDB
Inserted stats from team 7 into MongoDB
Inserted stats from team 8 into MongoDB
Inserted stats from team 9 into MongoDB
Inserted stats from team 10 into MongoDB
Inserted stats from team 11 into MongoDB
Inserted stats from team 14 into MongoDB
Inserted stats from team 15 into MongoDB
Inserted stats from team 16 into MongoDB
Inserted stats from team 17 into MongoDB
Inserted stats from team 19 into MongoDB
Inserted stats from team 20 into MongoDB
Inserted stats from team 21 into MongoDB
Inserted stats from team 22 into MongoDB
Inserted stats from team 23 into MongoDB
Inserted stats from team 24 into MongoDB
Inserted stats from team 25 into MongoDB
Inserted stats from team 26 into MongoDB
Inserted stats from team 27 into MongoDB
Inserted stats from team 28 into MongoDB
Inserted stats from team

In [4]:
# 4 Bringing in team name, from teams collection and joining to team stats

client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']
teams_collection = db['teams']
team_stats_collection = db['team_stats']

pipeline = [
    {
        '$lookup': {
            'from': 'teams',
            'localField': 'team_id',
            'foreignField': 'team.id',
            'as': 'team_info'
        }
    },
    {
        '$unwind': '$team_info'
    },
    {
        '$project': {
            '_id': 1,
            'season': 1,
            'team_id': 1,
            'stats': 1,
            'team_name': '$team_info.team.name'
        }
    }
]

# Execute the aggregation pipeline
result = list(team_stats_collection.aggregate(pipeline))

# Creating new collection with merged data
merged_collection = db['merged_team_stats']
merged_collection.insert_many(result)

print("Merge completed successfully!")

Merge completed successfully!


In [5]:
# 5 Converting stats to a per game measure

client = MongoClient('mongodb://localhost:27017/')
db = client['nba_data2']

# Collection
merged_collection = db['merged_team_stats']

# Fetch all documents from the collection
documents = merged_collection.find()

# Function to calculate per-game stats
def calculate_per_game(stats):
    games = stats['games']
    if games == 0:
        return stats  # Avoid division by zero
    per_game_stats = {
        "points avg": stats['points'] / games,
        "fgm": stats['fgm'] / games,
        "fga": stats['fga'] / games,
        "fgp": float(stats['fgp']) / 100,  # Assuming fgp is a percentage stored as a string
        "ftm": stats['ftm'] / games,
        "fta": stats['fta'] / games,
        "ftp": float(stats['ftp']) / 100,  # Assuming ftp is a percentage stored as a string
        "tpm": stats['tpm'] / games,
        "tpa": stats['tpa'] / games,
        "tpp": float(stats['tpp']) / 100,  # Assuming tpp is a percentage stored as a string
        "offReb": stats['offReb'] / games,
        "defReb": stats['defReb'] / games,
        "totReb": stats['totReb'] / games,
        "assists": stats['assists'] / games,
        "pFouls": stats['pFouls'] / games,
        "steals": stats['steals'] / games,
        "turnovers": stats['turnovers'] / games,
        "blocks": stats['blocks'] / games,
        "plusMinus": stats['plusMinus']  # Not per game
    }
    return per_game_stats

# Update each document with per-game statistics
for doc in documents:
    stats = doc['stats']
    per_game_stats = calculate_per_game(stats)
    
    # Create an update dictionary
    update_dict = {
        'team_name': doc['team_name'],
        'season': doc['season'],
        'games_stats': stats['games']
    }
    update_dict.update(per_game_stats)
    
    # Perform the update
    merged_collection.update_one(
        {'_id': doc['_id']},
        {'$set': update_dict}
    )

print("Update completed successfully!")

Update completed successfully!
